## Imports

In [ ]:
%env CUDA_VISIBLE_DEVICES=1

In [ ]:
import torch
import matplotlib.pyplot as plt
import pandas as pd
import os

In [ ]:
import matplotlib
matplotlib.rcParams['figure.facecolor'] = 'white'

In [ ]:
pd.options.display.max_columns = None

## Load data

In [ ]:
%run ../datasets/__init__.py

In [ ]:
dataset_kwargs = {
    'dataset_name': 'iu-x-ray',
    'dataset_type': 'all',
    'max_samples': None,
}
dataloader = prepare_data_classification(**dataset_kwargs)
dataset = dataloader.dataset
len(dataset)

## Compare runtime chexpert vs holistic

### Load holistic chexpert

In [ ]:
%run ../datasets/iu_xray.py

In [ ]:
fpath = os.path.join(DATASET_DIR, 'reports', 'reports_with_chexpert_labels.csv')
df = pd.read_csv(fpath, index_col=0)
df.replace(-1, 1, inplace=True)
df.replace(-2, 0, inplace=True)
df.head()

### Calculate light-labeler chexpert

In [ ]:
%run ../metrics/report_generation/labeler_correctness/light_labeler.py
%run ../utils/nlp.py

In [ ]:
labeler = ChexpertLightLabeler(dataloader.dataset.get_vocab())
labeler

In [ ]:
report_reader = ReportReader(dataloader.dataset.get_vocab())

In [ ]:
reports = list(df['Reports'])
reports = [
    report_reader.text_to_idx(report)
    for report in reports
]
len(reports)

In [ ]:
%%time

labels = labeler(reports)
labels.shape

In [ ]:
labels[labels == -2] = 0
labels[labels == -1] = 1
labels

### Calculate with full-labeler

In [ ]:
%run ../metrics/report_generation/labeler_correctness/full_labeler.py
%run ../utils/nlp.py

In [ ]:
labeler = ChexpertFullLabeler(dataloader.dataset.get_vocab())
labeler

In [ ]:
report_reader = ReportReader(dataloader.dataset.get_vocab())

In [ ]:
reports = list(df['Reports'])
reports = [
    report_reader.text_to_idx(report)
    for report in reports
]
len(reports)

In [ ]:
%%time

labels = labeler(reports)
labels.shape

In [ ]:
labels[labels == -2] = 0
labels[labels == -1] = 1
labels

### Calculate metrics between runtime and holistic

In [ ]:
%run ../metrics/report_generation/chexpert.py
%run -n ../eval_report_generation_chexpert_labeler.py

In [ ]:
def add_suffix(col):
    if col in CHEXPERT_LABELS:
        return f'{col}-gt'
    return col
df.rename(
    columns=add_suffix,
    inplace=True,
)
df.head()

In [ ]:
columns = labels_with_suffix('gen')
full_df = pd.concat([df, pd.DataFrame(labels, columns=columns)], axis=1)
full_df

In [ ]:
acc, precision, recall, f1, roc_auc = _calculate_metrics(full_df)
acc, precision, recall, f1, roc_auc

In [ ]:
roc_auc